In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [5]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [7]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [9]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [11]:
dataset=dataset.drop("User ID",axis=1)

In [13]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [15]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [17]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [19]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [21]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [23]:
dependent=dataset[['Purchased']]

In [25]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [29]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
from sklearn.model_selection import GridSearchCV
param_grid={'n_estimators': [50, 100, 200, 300],'criterion': ['gini', 'entropy', 'log_loss']}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')
                    

grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'n_estimators': [50, 100, 200, 300]},
             scoring='f1_weighted', verbose=3)

In [37]:
re=grid.cv_results_
grid_prediction=grid.predict(x_test)

In [39]:
from sklearn. metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

In [41]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

In [43]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The f1_macro value for best parameter{}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter{'criterion': 'gini', 'n_estimators': 300}: 0.9166666666666666


In [45]:
print("The confusion_matrix:\n",cm)

The confusion_matrix:
 [[74  5]
 [ 5 36]]


In [47]:
print ("The classification_report:\n",clf_report)

The classification_report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94        79
           1       0.88      0.88      0.88        41

    accuracy                           0.92       120
   macro avg       0.91      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [49]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9660389008953381

In [51]:
table=pd.DataFrame.from_dict(re)

In [53]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.218607,0.007755,0.028086,0.001926,gini,50,"{'criterion': 'gini', 'n_estimators': 50}",0.874254,0.857143,0.859435,0.947015,0.964286,0.900426,0.045798,2
1,0.421670,0.027729,0.046995,0.006863,gini,100,"{'criterion': 'gini', 'n_estimators': 100}",0.874254,0.838326,0.841398,0.911105,0.964286,0.885874,0.047242,8
2,0.814610,0.014223,0.058182,0.002417,gini,200,"{'criterion': 'gini', 'n_estimators': 200}",0.874254,0.857143,0.823129,0.911105,0.964286,0.885983,0.048337,7
3,1.206363,0.009888,0.076051,0.004224,gini,300,"{'criterion': 'gini', 'n_estimators': 300}",0.874254,0.875644,0.859435,0.911105,0.982051,0.900498,0.044173,1
4,0.212313,0.007117,0.026746,0.001274,entropy,50,"{'criterion': 'entropy', 'n_estimators': 50}",0.874254,0.840114,0.859435,0.911105,0.927778,0.882537,0.032450,9
5,0.406407,0.006851,0.036974,0.003289,entropy,100,"{'criterion': 'entropy', 'n_estimators': 100}",0.874254,0.875644,0.841398,0.911105,0.963889,0.893258,0.041637,3
6,0.818928,0.008488,0.056275,0.002261,entropy,200,"{'criterion': 'entropy', 'n_estimators': 200}",0.874254,0.857143,0.841398,0.911105,0.963889,0.889558,0.043803,5
7,1.255024,0.028551,0.086153,0.009324,entropy,300,"{'criterion': 'entropy', 'n_estimators': 300}",0.874254,0.875644,0.823129,0.911105,0.927778,0.882382,0.036078,10
8,0.206016,0.004157,0.027193,0.001476,log_loss,50,"{'criterion': 'log_loss', 'n_estimators': 50}",0.892857,0.838326,0.841398,0.892857,0.926743,0.878436,0.033853,12
9,0.423977,0.008495,0.042089,0.009155,log_loss,100,"{'criterion': 'log_loss', 'n_estimators': 100}",0.874254,0.857143,0.841398,0.911105,0.927778,0.882336,0.032461,11


In [55]:
age_input=int(input("Age:"))
bmi_input=int(input("BMI:"))
children_input=int(input("Children:"))
sex_male_input=int(input("Sex_male 0 or 1:"))
smoker_yes__input=int(input("Smoker_yes 0 or 1:"))

Age: 32
BMI: 43
Children: 2
Sex_male 0 or 1: 0
Smoker_yes 0 or 1: 1


In [57]:
Future_prediction=grid.predict([[age_input,bmi_input,children_input]])

In [59]:
Future_prediction

array([1], dtype=int64)